In [48]:
import torch
import torchvision

model = torchvision.models.resnet18(weights="IMAGENET1K_V1")

resnet18_image = torch.rand(1, 3, 224, 224)

torch.onnx.export(model, resnet18_image, "./resnet18/resnet18.onnx")

In [49]:
import tensorrt as trt

class TensorRTConversion:
    """
    	path: to onnx
    	path: to engine
    	maxworkspace: gb < lgb
    	precision: 16 float and half precision
    	Inference mode: Dynamic Batch [1, 10, 20]
	"""
    def __init__(self, path_to_onnx, path_to_engine, max_workspace_size=1 << 30, half_precision=False):
        self.TRT_LOGGER = trt.Logger(trt.Logger.WARNING)
        self.path_to_onnx = path_to_onnx
        self.path_to_engine = path_to_engine
        self.max_workspace_size = max_workspace_size
        self.half_precision = half_precision
    """
        { INIT BUILD
        INIT CONFIG
        INIT EXPLICIT BATCH
        INIT NETWORK }
        Tensorrt >= 8.0.0
    """
    def convert(self):
        builder = trt.Builder(self.TRT_LOGGER)
        config = builder.create_builder_config()

        # config.max_workspace_size = self.max_workspace_size
        config.set_memory_pool_limit(trt.MemoryPoolType.WORKSPACE, 1 << 30) 
        
        explicit_batch = 1<< int(trt.NetworkDefinitionCreationFlag.EXPLICIT_BATCH)

        network = builder.create_network(explicit_batch)

        parser = trt.OnnxParser(network, self.TRT_LOGGER)

        with open(self.path_to_onnx, 'rb') as model_onnx:
            if not parser.parse(model_onnx.read()):
                print("ERROR: Failed to parse ONNX Model")
                for error in parser.errors:
                    print(error)
                return None

        # Set profile for explicit batch
        # profile = builder.create_optimization_profile()
        # profile.set_shape("input_name", min=(1, 3, 224, 224), opt=(10, 3, 224, 224), max=(20, 3, 224, 224))
        # config.add_optimization_profile(profile)
        print("Successfully TensorRT Engine Configured to Max Batch")
        print("\n")

        if builder.platform_has_fast_fp16:
            config.set_flag(trt.BuilderFlag.FP16)

        engine = builder.build_serialized_network(network, config)

        with open(self.path_to_engine, "wb") as f_engine:
            f_engine.write(engine)

        print("Successfully Convert ONNX to TensorRT Dynamic Engine")
        print(f"Serialized engine saved in engine path: {self.path_to_engine}")

# Init class
convert = TensorRTConversion("./resnet18/resnet18.onnx", "./resnet18/resnet18.engine")
# Call class method
convert.convert()

Successfully TensorRT Engine Configured to Max Batch


Successfully Convert ONNX to TensorRT Dynamic Engine
Serialized engine saved in engine path: ./resnet18/resnet18.engine


In [50]:
import pycuda.autoinit
import pycuda.driver as cuda
import cv2, os
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from pathlib import Path

In [51]:
"""
class name: TRTInference
INIT: self.logger
params [1]: engine_path, onnx_path, context
params [2]: input shape and output shape, class_labels
"""
class TRTInference:
    def __init__(self, engine_file_path, input_shape, output_shape, class_labels_file):
        self.logger = trt.Logger(trt.Logger.WARNING)
        self.engine_file_path = engine_file_path
        
        # Load engine
        self.engine = self.load_engine(self.engine_file_path)
        
        # Init context
        self.context = self.engine.create_execution_context()
        
        self.input_shape = input_shape
        self.output_shape = output_shape
        self.class_labels_file = class_labels_file

        # Open class file
        with open(class_labels_file, "r") as class_read:
            self.class_labels = [line.strip() for line in class_read.readlines()]

    def load_engine(self, engine_file_path):
        with open(engine_file_path, "rb") as f:
            runtime = trt.Runtime(self.logger)
            engine_deserialized = runtime.deserialize_cuda_engine(f.read())
        return engine_deserialized

    """
    param [1]: image path
    results(return): img_list, img_path
    img resolution: 1, 3, 224, 224
    """
    def preprocess_img(self, image_path):
        img_list = []
        img_path = []
        for img_original in os.listdir(image_path):
            if img_original.endswith(".jpg") or img_original.endswith(".png") or img_original.endswith(".jpeg"):
                img_full_path = os.path.join(image_path, img_original)

                # Open image
                image = Image.open(img_full_path)

                # Resize image
                # [224, 224]
                img = image.resize((self.input_shape[2], self.input_shape[3]), Image.NEAREST)
                img_np = np.array(img).astype(np.float32) / 255.0
                img_np = img_np.transpose((2, 0, 1))
                img_np = np.expand_dims(img_np, axis=0)
                img_list.append(img_np)
                img_path.append(img_full_path)
        return img_list, img_path

    # Process labels with corresponding images
    def postprocess_img(self, outputs):
        # Classes
        classes_indices = []

        for output in outputs:
            class_idx = output.argmax()
            print("Class detected:", self.class_labels[class_idx])
            classes_indices.append(self.class_labels[class_idx])
        return classes_indices

    # Inference detection
    """
    param [0] = self
    param [1] = image_path
    target: Inference detection on GPU local
    """
    def inference_detection(self, image_path):
        # List
        input_list, full_img_paths = self.preprocess_img(image_path)
        results = []
        for inputs, full_img_path in zip(input_list, full_img_paths):
            inputs = np.ascontiguousarray(inputs)
            outputs = np.empty(self.output_shape, dtype=np.float32)

            d_inputs = cuda.mem_alloc(1 * inputs.nbytes)
            d_outputs = cuda.mem_alloc(1 * outputs.nbytes)
            bindings = [d_inputs, d_outputs]
            
            # Transfer input to GPU
            cuda.memcpy_htod(d_inputs, inputs)
            
            # Synchronize
            self.context.execute_v2(bindings)
            
            # Copy output back to host (CPU)
            cuda.memcpy_dtoh(outputs, d_outputs)
            result = self.postprocess_img(outputs)
            d_inputs.free()
            d_outputs.free()
            # Results
            results.append(result)
            # Display results
            self.display_recognized_images(full_img_path, result)
        return results

    """
    param[0]: image_path
    param[1]: class_label
    targetL Display and save detected images
    """
    def display_recognized_images(self, image_path, class_label):
        image = Image.open(image_path)
        for class_name in class_label:
            # Create one directory for detected images
            path_to_detected_imgs = "./resnet18/images_detected"

            # Check path existence
            if not os.path.exists(path_to_detected_imgs):
                os.makedirs(path_to_detected_imgs)

            plt.imshow(image)
            plt.title(f"Recognized Image: {class_name}")
            plt.axis("off")
            # image_name = os.path.basename(image_path)
            image_name = Path(image_path).stem
            save_img = os.path.join(path_to_detected_imgs, f"{class_name}_{image_name}.jpg")
            plt.savefig(save_img)
            plt.close()
            return image

engine_file_path = "./resnet18/resnet18.engine"
input_shape = (1, 3, 640, 640)
output_shape = (1, 1000)
class_labels = "./resnet18/imagenet-classes.txt"
path_to_org_images = "./resnet18/images"

inference = TRTInference(engine_file_path, input_shape, output_shape, class_labels)
inference.inference_detection(path_to_org_images)

Class detected: ballpoint, ballpoint pen, ballpen, Biro
Class detected: window screen
Class detected: redshank, Tringa totanus
Class detected: window shade
Class detected: window shade
Class detected: sandbar, sand bar
Class detected: window shade


[['ballpoint, ballpoint pen, ballpen, Biro'],
 ['window screen'],
 ['redshank, Tringa totanus'],
 ['window shade'],
 ['window shade'],
 ['sandbar, sand bar'],
 ['window shade']]

In [52]:
# I guess there is problem about image preprocessing so the inference result is so bad
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)

img = Image.open('./resnet18/images/images-1.jpeg').convert("RGB")
img = torchvision.transforms.ToTensor()(img)
images = [img.to(device)]

model.eval()

predictions = model(images)
pred = predictions[0]
print(pred)

TypeError: conv2d() received an invalid combination of arguments - got (list, Parameter, NoneType, tuple, tuple, tuple, int), but expected one of:
 * (Tensor input, Tensor weight, Tensor bias, tuple of ints stride, tuple of ints padding, tuple of ints dilation, int groups)
      didn't match because some of the arguments have invalid types: (!list of [Tensor]!, !Parameter!, !NoneType!, !tuple of (int, int)!, !tuple of (int, int)!, !tuple of (int, int)!, int)
 * (Tensor input, Tensor weight, Tensor bias, tuple of ints stride, str padding, tuple of ints dilation, int groups)
      didn't match because some of the arguments have invalid types: (!list of [Tensor]!, !Parameter!, !NoneType!, !tuple of (int, int)!, !tuple of (int, int)!, !tuple of (int, int)!, int)
